In [1]:
#----------------------------#
# IDEA
#----------------------------#
# Get Submission PCA Data Structures 
# 1.) Create embedding dictionarys 
# 2.) Average embedding for subreddits 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import Counter
import import_ipynb
import math

In [2]:
home_path = %pwd
data_path = "/home/pavel/Documents/Keusch_Job/Data/"
subroutines_path = "/home/pavel/Documents/Keusch_Job/Subroutines/"

In [3]:
#Subroutines
%cd  $subroutines_path
%run "selected_subreddits.ipynb"
%run "data_loader.ipynb"
%run "basic_plots.ipynb"
%cd  $home_path

/home/pavel/Documents/Keusch_Job/Subroutines
/home/pavel/Documents/Keusch_Job/Tasks/create_datastructures


In [4]:
#----------------------------#
# 1.) Create embedding dictionaries
#----------------------------#

In [5]:
import tensorflow as tf
import tensorflow_hub as hub

In [6]:
module_url = data_path + "universal-sentence-encoder_4"
embed_model = hub.load(module_url)

In [7]:
def embed(input): return embed_model(input)
def tf_simil(s1,s2): return np.inner(s1,s2) 
def tf_simil_matrix(embeddings, rd = 8): return [[round(tf_simil(s1,s2),rd) for s1 in embeddings] for s2 in embeddings]

In [11]:
def embed_dict_create(df, key_var="id", text_var="title"):
    #df: something like S, with an id and titles
    embed_dict = {}
    for i in range(0,len(df)):
        print(str(i) + " : " + df[key_var].iloc[i])
        embed_dict[df[key_var].iloc[i]] = embed([df[text_var].iloc[i]])
    return(embed_dict)

In [23]:
#TEST RUN
#path= data_path
#i = 1
#S_1 = load_data(data_name="SUBM_2020_0" + str(i) + ".csv", data_path = path)
#S_1  = S_1.merge(selected_subreddits[["subreddit","subr_id"]], how="inner",
#                               left_on = "subreddit",right_on = "subreddit")
#embed_dict_1 = embed_dict_create(S_1, key_var="id", text_var ="title")
#np.save(data_path + 'embed_dict_1.npy', embed_dict_1)

In [15]:
def embed_month(month_index=1, path = data_path):
    S = load_data(data_name="SUBM_2020_0" + str(month_index) + ".csv", data_path = path)
    S  = S.merge(selected_subreddits[["subreddit","subr_id"]], how="inner",
                                   left_on = "subreddit",right_on = "subreddit")
    embed_dict = embed_dict_create(S, key_var="id", text_var ="title")
    np.save(data_path + 'embed_dict_' + str(month_index) + '.npy', embed_dict)

In [ ]:
#LOOP FOR EVERY MONTH
for i in range(1:6):
    embed_month(month_index = i, path = data_path)

In [ ]:
#----------------------------#
# 2.) Create embedding dicitonaries
#----------------------------#

In [8]:
def avg_emb_from_dict(key_list,embed_dict):
    # averages of vectors should be allowed IF this is a vector
    # https://www.tf.uni-kiel.de/matwis/amat/admat_en/kap_2/basics/b2_1_1.html
    # https://math.stackexchange.com/questions/80923/average-of-multiple-vectors
    out = embed_dict[key_list[0]]
    for i in range(1,len(key_list)):
        out = out + embed_dict[key_list[i]]
    return(out/len(key_list))

In [18]:
#TEST RUN
#path = data_path
#month_index = 1
#Load Data
#S = load_data(data_name="SUBM_2020_0" + str(month_index) + ".csv", data_path = path)
#S  = S.merge(selected_subreddits[["subreddit","subr_id"]], how="inner",
#                                   left_on = "subreddit",right_on = "subreddit")
#Create Data Structures
#subr_list = S["subreddit"].unique().tolist() 
#submission_df = S[["subreddit","id"]].copy()
#del S
#embed_dict = np.load(path + 'embed_dict' + str(month_index) + '.npy',allow_pickle='TRUE').item()
#Testwise just using this
#embed_dict = np.load(path + 'embed_dict.npy',allow_pickle='TRUE').item()
#
#
#Run
#avg_emb_dict = {}
#for i in range(0,len(subr_list)):
#    current_subr_keys = submission_df["id"].loc[submission_df["subreddit"]==subr_list[i],].tolist()
#    avg_emb_dict[subr_list[i]] = avg_emb_from_dict(current_subr_keys,embed_dict)
#Save
#np.save(data_path + 'avg_emb_dict' + str(month_index) + '.npy', avg_emb_dict)

In [20]:
def avg_submission_emb_from_dict(month_index= 1, path = data_path):
    #Load Data
    S = load_data(data_name="SUBM_2020_0" + str(month_index) + ".csv", data_path = path)
    S  = S.merge(selected_subreddits[["subreddit","subr_id"]], how="inner",
                                   left_on = "subreddit",right_on = "subreddit")
    #Create Data Structures
    subr_list = S["subreddit"].unique().tolist() 
    submission_df = S[["subreddit","id"]].copy()
    del S
    embed_dict = np.load(path + 'embed_dict_' + str(month_index) + '.npy',allow_pickle='TRUE').item()
    #Run
    avg_emb_dict = {}
    for i in range(0,len(subr_list)):
        current_subr_keys = submission_df["id"].loc[submission_df["subreddit"]==subr_list[i],].tolist()
        avg_emb_dict[subr_list[i]] = avg_emb_from_dict(current_subr_keys,embed_dict)
    #Save
    np.save(data_path + 'avg_emb_dict' + str(month_index) + '.npy', avg_emb_dict)

In [ ]:
for i in range(1:6).
    avg_submission_emb_from_dict(i,data_path)